In [ ]:
%matplotlib inline
#%matplotlib notebook
import os
import matplotlib
if not os.getenv("DISPLAY"): matplotlib.use('Agg')
import matplotlib.pylab as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels \
    import RBF, WhiteKernel, RationalQuadratic, ExpSineSquared, Matern, ConstantKernel
import pickle, gzip
import itertools
from timeit import default_timer as timer
import resource, sys

In [ ]:
G = pd.read_csv('../../data/gem_uq_inoutput.csv')
G.describe().T

In [ ]:
G.te_value = np.log10(G.te_value)
G.ti_value = np.log10(G.ti_value)
G.te_ddrho = np.log10(-G.te_ddrho)
G.ti_ddrho = np.log10(-G.ti_ddrho)
G.te_transp_flux = np.log10(G.te_transp_flux)
G.ti_transp_flux = np.log10(G.ti_transp_flux)
G.describe().T

In [ ]:
axes = pd.plotting.scatter_matrix(G, alpha=0.5, diagonal='hist', figsize=(8,6))
corr = np.array(G.corr())
for i, j in zip(*plt.np.triu_indices_from(axes, k=1)):
    axes[i, j].annotate("%.3f" %corr[i,j], (0.8, 0.8), xycoords='axes fraction', ha='center', va='center')
plt.show()

In [ ]:
XLABELS = ['te_value', 'ti_value', 'te_ddrho', 'ti_ddrho']
X = G[XLABELS].values
YLABELS = ['te_transp_flux', 'ti_transp_flux']
Y = G[YLABELS].values

In [ ]:
NX = len(XLABELS)
NY = len(YLABELS)
print('Number of points = ', X.shape[0])

kernel = ConstantKernel() + Matern(length_scale=2, nu=3/2) + WhiteKernel(noise_level=1)

start=timer()
GP = []
for F in range(NY): 
    gp = GaussianProcessRegressor(kernel=kernel, normalize_y=True)
#    %time 
    gp.fit(X, Y[:,F])
    print(YLABELS[F], gp.kernel_)
    GP.append(gp)
end=timer()
print('Calculating GPR took %0.3f seconds' % (end-start))


In [ ]:
start=timer()
Xt = X.copy()
Yt = np.zeros((Xt.shape[0],NY))
Yts = np.zeros((Xt.shape[0],NY))
for F in range(NY):
  Yt[:,F], Yts[:,F] = GP[F].predict(Xt, return_std=True)
end=timer()
end=timer()
print('Calculating GPR on grid took %0.3f seconds' % (end-start))

In [ ]:
np.sqrt((Y-Yt)**2).mean(axis=0)

In [ ]:
Yts.mean(axis=0)

In [ ]:
Xvals = np.array([np.unique(X[:,i]) for i in range(X.shape[1])])

In [ ]:
def vis(slice):
    if slice == 0:
        Xt = np.array([i for i in itertools.product(np.linspace(Xvals[0].min(), Xvals[0].max(), 100), 
                                                    [Xvals[1,2]], 
                                                    [Xvals[2,2]],
                                                    [Xvals[3,2]])])
    elif slice == 1:
        Xt = np.array([i for i in itertools.product([Xvals[0,2]], 
                                                    np.linspace(Xvals[1].min(), Xvals[1].max(), 100), 
                                                    [Xvals[2,2]],
                                                    [Xvals[3,2]])])
    elif slice == 2:
        Xt = np.array([i for i in itertools.product([Xvals[0,2]], 
                                                    [Xvals[1,2]],
                                                    np.linspace(Xvals[2].min(), Xvals[2].max(), 100), 
                                                    [Xvals[3,2]])])
    elif slice == 3:
        Xt = np.array([i for i in itertools.product([Xvals[0,2]], 
                                                    [Xvals[1,2]],
                                                    [Xvals[2,2]],
                                                    np.linspace(Xvals[3].min(), Xvals[3].max(), 100))])
    start=timer()
    Yt = np.zeros((Xt.shape[0],NY))
    Yts = np.zeros((Xt.shape[0],NY))
    for F in range(NY):
      Yt[:,F], Yts[:,F] = GP[F].predict(Xt, return_std=True)
    end=timer()
    print('Calculating GPR on grid took %0.3f seconds' % (end-start))
    
    plt.figure(figsize=(8,6))
    plt.plot(10.0**(Xt[:,slice]), 10.0**(Yt[:,0]), 'r', label=YLABELS[0])
    plt.fill_between(10.0**(Xt[:,slice]), 10.0**(Yt[:,0]-Yts[:,0]), 10.0**(Yt[:,0]+Yts[:,0]), color='red', alpha=0.5)
    plt.plot(10.0**(Xt[:,slice]), 10.0**(Yt[:,1]), 'b', label=YLABELS[1])
    plt.fill_between(10.0**(Xt[:,slice]), 10.0**(Yt[:,1]-Yts[:,1]), 10.0**(Yt[:,1]+Yts[:,1]), color='blue', alpha=0.5)
    plt.xlabel(XLABELS[slice])
    plt.legend(loc=0)

In [ ]:
for i in range(NX):
    vis(i)